In [1]:
# This file contains codes of the lecture notes and homeworks for the course of "computaional methods for data
# analysis"

# %pylab inline
%pylab 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [ ]:
# lecture 1&2....Reconstructing a noisy data by filtering and averaging noise around the main 
#frequency by taking 
# fft of the original signal in time 

T=30
n=512
t2=linspace(-T/2,T/2,n+1)
t=t2[0:n]
u=1/cosh(t)  #1/cosh = sech

k = (2*pi/T)*concatenate([arange(n/2),arange(-n/2,0,1)])
ks = fft.fftshift(k)

ut = fft.fft(u)     #it seems if you call the module name once, it will be enough
noise = 20
utn = ut + noise*(random.randn(n) + 1j*random.randn(n))

un = abs(fft.ifft(utn))

fltr = exp(-(k+5)**2)
utnf = utn*fltr
unf = abs(fft.ifft(utnf))

utavg=zeros(n,dtype=complex)    # this block shows how white noise converges to zero and original signal appears
                                # by taking more samples
for i in range(100):
    utn = ut + noise*(random.randn(n)+1j*random.randn(n))
    utavg=utavg+utn
    utavg2=abs(fft.fftshift(utavg))/(i+1)
    cla()
    plot(ks,abs(fft.fftshift(ut)),'-b',ks,utavg2,'-r')
    pause(0.2)


# subplot(3,1,1)
# plot(t,u,'-b',t,un,'-g',t,unf,'-m',t,0.5*ones(len(t)),'--k')

# subplot(3,1,2)
# plot(ks,abs(fft.fftshift(ut))/max(abs(ut)),'-r',\
#      ks,abs(fft.fftshift(utn))/max(abs(utn)),'-m',\
#      ks,fft.fftshift(fltr),'-k')
# xlim([-10,10])

# subplot(3,1,3)
# plot(ks,fft.fftshift(angle(ut)))
# xlim([-2,2])

#plot(t,abs(fft.ifft(ut)),'--r')



In [ ]:
# lecture 3
# To show that a translation of time signal does not change the frequency spectrum and fft components remain 
# the same.

T=60
n=512
t2=linspace(-T/2,T/2,n+1)
t=t2[0:n]

s = linspace(0,10,21)
Ts,S = meshgrid(t,s)

U = 1/cosh(Ts-10*sin(S))

k = (2*pi/T)*concatenate([arange(n/2),arange(-n/2,0,1)])
ks = fft.fftshift(k)

[Ks,S] = meshgrid(ks,s)

noise=10
Ut = zeros((len(s),n),dtype=complex)
Un = zeros((len(s),n),dtype=complex)
for i in range(len(s)):
    Ut[i,:]=fft.fft(U[i,:])+noise*(random.randn(n)+1j*random.randn(n))
    Un[i,:]=fft.ifft(Ut[i,:])
    
from mpl_toolkits.mplot3d import axes3d
fig = figure()
ax = fig.add_subplot(211, projection='3d')
ax.plot_wireframe(Ts,S,abs(fft.fftshift(Un)),rstride=1,cstride=0) 
ax.view_init(elev=75,azim=-60)

ax = fig.add_subplot(212, projection='3d')
ax.plot_wireframe(Ks,S,abs(fft.fftshift(Ut)),rstride=1,cstride=0) 
ax.view_init(elev=75,azim=-60)



In [ ]:
# HW 1: Determine the trajectory of marble inside the dog's intestine
# could not finish, hard to figure out how to take fft of 3d data
import scipy.io as sio

mat_file = sio.loadmat('Testdata.mat')    # import matlab file data Testdata.mat
sorted(mat_file.keys())                   # lists all components such as variable names in the file

Undata = mat_file['Undata']                # Undata is the array of data revealed by previous sorted command
                                          # data shape is (20,262144)
L = 15    # given spatial length
n = 64    # given Fourier modes
x2 = linspace(-L,L,n+1)
x = x2[0:n]
y = x
z = x

k = 2*pi/(2*L)*concatenate([arange(n/2),arange(-n/2,0,1)])
ks = fft.fftshift(k)

X,Y,Z = meshgrid(x,y,z)
KX,KY,KZ = meshgrid(ks,ks,ks)

for i in range(20):
    Un = reshape(Undata[i,:],(n,n,n),'F')
    cla()
    

# # averagaing the signal data:
# avg = array(zeros((n,n,n)),dtype=complex))
# for i in range(data.shape[0]):
#     ut = fft.fftn(reshape(Undata[i,:],(n,n,n),'F'))
#     avg = avg + ut
#     avg2 = avg/(i+1)
#     cla()
#     plot(abs(avg2))
#     pause(0.2)



In [ ]:
#Week 3, Lecture 7.... Multi resolution analysis... By changing the width of the filtering window in time, build a 
# spectrogram (frequency, time) multi-resolution graph of the time signal

from mpl_toolkits.mplot3d import axes3d

L=10
n=2048
t2=linspace(0,L,n+1)
t=t2[0:n]

S= 3*sin(2*t)+0.5*tanh(0.5*(t-3))-0.2*exp(-(t-4)**2)+ \
                       1.5*sin(5*t)+0.4*cos(3*(t-6)**2)+(t/20)**3 

k=(2*pi/L)*concatenate([arange(n/2),arange(-n/2,0,1)])
ks=fft.fftshift(k)

width=array([10,1,0.2])
slice=linspace(0,10,101)

fig1,ax1=subplots(3)
Sgtv=zeros((len(slice),n))
for i in range(len(slice)):
    
    g=exp(-width[0]*(t-slice[i])**2)
    Sg=S*g
    Sgt=fft.fft(Sg)
    Sgtv[i,:]=abs(fft.fftshift(Sgt))
    ax1[0].cla()
    ax1[0].plot(t,S,t,g)
    ax1[1].cla()
    ax1[1].plot(t,Sg)
    ax1[2].cla()
    ax1[2].plot(ks,abs(fft.fftshift(Sgt))/max(abs(Sgt)))
    xlim(-60,60)
    
    pause(0.1)
    
Ks,Slice=meshgrid(ks,slice)

# close(fig1)
fig2=figure()
ax2=fig2.gca(projection='3d')
ax2.plot_surface(Slice[:,923:1123],Ks[:,923:1123],Sgtv[:,923:1123],cmap='gist_earth',rstride=5,cstride=5)
# ax2.set_ylim(-60,60)
ax2.view_init(elev=90,azim=0) 

In [ ]:
#Week 4, Lecture 9 and 10 ....... To reduce the size of the image(data) and reconstruct it using filtering 
# the fft content

from skimage.color import rgb2gray

pathtophoto = "/home/ahmad/Pictures/Sharbat_Gula.jpg"
A = imread(pathtophoto)
# imshow(A)

Abw = rgb2gray(A)   # Abw.shape is (400,254) and is float
# imshow(Abw)
    
noise = 0.5
Abw = Abw + noise*randn(400,254)
figure()
imshow(Abw)

Abwt = fftshift(fft.fft2(Abw))
# figure()
# imshow(log(abs(Abwt)))

kx = arange(1,255)
ky = arange(1,401)
Kx,Ky = meshgrid(kx,ky)

# width = 0.01
# F = exp(-width*(Kx-128)**2-width*(Ky-201)**2)    # Gaussian filtering

width = 40
length=20
F = zeros((400,254))
F[201-width:201+width,127-length:127+length]=ones((2*width,2*length))    # step function filtering
# figure()
# imshow(F)

Abwtf = Abwt*F
Abw_reconstructed = abs(ifft2(ifftshift(Abwtf)))
figure()
imshow(Abw_reconstructed)

In [ ]:
## ------- Week 4 Lecture 11, cleaning up an image with diffusion
from skimage.color import rgb2gray
from scipy.integrate import solve_ivp

A = imread("/home/ahmad/Pictures/Sharbat_Gula.jpg")

Abw = rgb2gray(A)
nx,ny = Abw.shape

noise = 0.5
An = Abw + noise*randn(nx,ny)

from scipy.sparse import spdiags,kron
x = linspace(0,1,nx)
dx = x[1]-x[0]
y = linspace(0,1,ny)
dy = y[1]-y[0]

e1x = ones(nx)
e1y = ones(ny)
Dx = spdiags([e1x,-2*e1x,e1x],[-1,0,1],nx,nx) / dx**2  # second derivative matrix in one dimension 
Dy = spdiags([e1y,-2*e1y,e1y],[-1,0,1],ny,ny) / dy**2

Ix = eye(nx)
Iy = eye(ny)
L = kron(Iy,Dx) + kron(Dy,Ix)     # constructs laplacian in two dimension using kron command

D = 0.0005   # diffusion constant
u0 = reshape(An,(nx*ny),'F')    # 'F' means fortran-like (as matlab), if C-like, needs to change L

def image_rhs(t,u,D,L):
    return D*L.dot(u)

T = 0.1
sol = solve_ivp(image_rhs,[0,T],u0,t_eval=arange(0,T,T/10),args=(D,L))   # add more options

for i in range(len(sol.t)):
    imshow(reshape(sol.y[:,i],(nx,ny),'F'))
    pause(0.2)

In [ ]:
# week 5&6 Lectures 13-16, first example
from mpl_toolkits.mplot3d import axes3d

x = linspace(0,1,25)
t = linspace(0,2,50)

T,X = meshgrid(t,x)     # if you change the order of X & T, the interpreration of u and vh would be different
f = exp(-abs((X-0.5)*(T-1))) + sin(X*T)

fig = figure()
ax = fig.add_subplot(2,2,1,projection = '3d')
ax.plot_surface(X,T,f,cmap='gist_earth')

u,s,vh = linalg.svd(f,full_matrices=False)

u.shape,s.shape,vh.shape
figure()
plot(s,'ro')

for i in range(3):
    ff = dot(u[:,:i+1]*s[:i+1],vh[:i+1,:])      #ff = dot(dot(u[:,:i+1],diag(s[:i+1])),vh[:i+1,:])
    ax = fig.add_subplot(2,2,i+2,projection='3d')
    ax.plot_surface(X,T,ff,cmap='gist_earth')
    
energy_ratio = sqrt(sum(s[:2]**2))/sqrt(sum(s**2))  # energy portion of modes used to reconstruct f
        
figure()
plot(x,u[:,0],'b-',x,u[:,1],'r-',x,u[:,2],'-g')    # first three spatial modes used to reconstruct f


In [ ]:
# week 5&6 Lectures 13-16, first example

from mpl_toolkits.mplot3d import axes3d

x = linspace(-10,10,100)
t = linspace(0,10,30)

X,T = meshgrid(x,t)

f = (1-0.5*cos(2*T))/cosh(X) + (1-0.5*sin(2*T))*tanh(X)/cosh(X)

fig1 = figure()
ax1 = fig1.add_subplot(2,2,1,projection='3d')
ax1.plot_wireframe(X,T,f,rstride=2,cstride=0)

u,s,vh = linalg.svd(f,full_matrices=False)

# figure()
# plot(s,'ro')

# for j in range(3):
#     ff = dot(u[:,:j+1]*s[:j+1],vh[:j+1,:])
#     ax = fig1.add_subplot(2,2,j+2,projection='3d')
#     ax.plot_wireframe(X,T,ff,rstride=2,cstride=0)
        
# figure()
# plot(x,vh[0,:],'b-',x,vh[1,:],'r-')     # main spatial modes
# figure()
# plot(t,u[:,0],t,u[:,1],t,u[:,2])        # main temporal modes

    
# energy_ratio = sqrt(sum(s[:2]**2))/sqrt(sum(s**2))



In [28]:
# HW 2. Principal Component Analysis---or SVD ... "Could not figure out how to work with this"
# see the following webpage: http://courses.washington.edu/amath582/582hw2.pdf

import scipy.io as sio                   # to import matlab data
from skimage.color import rgb2gray       # use gray frames to avoid dealing with extra dimension

cam11 = sio.loadmat("C:/Users/admin/Desktop/Ahmad/python_codes/comp_meth_data_anal_code_files/cam1_1.mat")
vidFrames1_1 = cam11['vidFrames1_1']
m,n,p,q = vidFrames1_1.shape
# for jj in range(q):                    # to watch the original movie
#     cla()
#     imshow(vidFrames1_1[:,:,:,jj])

# frame = zeros((q,m*n))
# for i in range(q):
#     frame[i,:] = reshape(rgb2gray(vidFrames1_1[:,:,:,i]),(m*n),'F')
    
# u,s,vh = linalg.svd(frame,full_matrices=False)
# figure()
# plot(s[:10],'ro')
# num_modes = 1

# for ii in range(num_modes):
#     fframe = dot(u[:,:ii+1]*s[:ii+1],vh[:ii+1,:])
    
# for jj in range(q):
#     cla()
#     imshow(reshape(fframe[jj,:],(m,n),'F'))
#     pause(0.1)


In [ ]:
fframe.shape,frame.shape,u.shape,s.shape,vh.shape

In [ ]:
# Week 7, Lecture 19.... implementing image separation using independent component analysis and svd
# we mix two photos of the same size and then using ICA and SVD we try to separate them to their original versions.
# you can change the value of beta to make different mixings of the original images.

S1 = imread("/home/ahmad/Pictures/Sharbat_Gula.jpg")
S2 = imread("/home/ahmad/Pictures/photo.jpg")
S2_r = S2[0:1536:3,0:2048:8,:]
S2 = S2_r[0:400,0:254,:]      # resize the photo to be the same as S1 (400,254,3)
m,n,v = S1.shape

beta = 2/5
A = array([[4/5,beta],[-2/5,3/4]])    # mixing matrix - mixes the two original photos

X1 = A[0,0]*S1 + A[0,1]*S2
X2 = A[1,0]*S1 + A[1,1]*S2

x1 = reshape(X1,(m*n*v),'F')    # do not have to reshape, but easy to appy the formula
x2 = reshape(X2,(m*n*v),'F')

x1 = x1-mean(x1)
x2 = x2-mean(x2)

theta0 = 0.5*arctan(-2*sum(x1*x2)/sum(x2**2-x1**2))    # theta0 is the angle of maximal/minimal variance...
                                                       # obtained from analytical formula
Ustar = array([[cos(theta0),sin(theta0)],[-sin(theta0),cos(theta0)]])  # Ustar is the first rotaion matrix

sigma1 = sum((cos(theta0)*x1+sin(theta0)*x2)**2)    # maximum directional variance
sigma2 = sum((cos(theta0-pi/2)*x1+sin(theta0-pi/2)*x2)**2)   # minimum directional variance

Sig = array([[1/sqrt(sigma1), 0],[0, 1/sqrt(sigma2)]])     # Sig is the inverse of Sigma matrix of SVD

x1bar = Sig[0,0]*(Ustar[0,0]*x1+Ustar[0,1]*x2)   # X1bar and X2bar are already processed using the two matrices...
x2bar = Sig[1,1]*(Ustar[1,0]*x1+Ustar[1,1]*x2)  # that have been calculated so far

phi0 = (1/4)*arctan(-sum((2*x1bar**3*x2bar-2*x1bar*x2bar**3)/(x1bar**2+x2bar**2))/ \
                    sum((3*x1bar**2*x2bar**2-0.5*x1bar**4-0.5*x2bar**4)/(x1bar**2+x2bar**2)))
                                            # phi0 is obtained by minimizing Kurtosis through analytical formula
V = array([[cos(phi0),sin(phi0)],[-sin(phi0),cos(phi0)]])

s1 = V[0,0]*x1bar+V[0,1]*x2bar
s2 = V[1,0]*x1bar+V[1,1]*x2bar

s1 = s1 - min(s1)          # to make it start from 0
s1 = s1*(255/max(s1))   # to make it from 0 - 255 appropriate for rgb values of pixels

s2 = s2 - min(s2)
s2 = s2*(255/max(s2))

S1_reconstructed = reshape(s1,(m,n,v),'F')
S2_reconstructed = reshape(s2,(m,n,v),'F')

figure()
subplot(3,2,1), imshow(S1)                       # original image
subplot(3,2,2), imshow(S2)
subplot(3,2,3), imshow(uint8(X1))                # mixed image
subplot(3,2,4), imshow(uint8(X2))
subplot(3,2,5), imshow(uint8(S1_reconstructed))  # separated image
subplot(3,2,6), imshow(uint8(S2_reconstructed))


In [ ]:
a = arange(10,21)
a
print((a-min(a))*(255/max(a)))

In [ ]:
-sum((2*X1bar**3*X2bar-2*X1bar*X2bar**3)/(X1bar**2+X2bar**2))


In [ ]:
A = fix(10*rand(5,5))
B = arange(3)
C = fix(10*rand(3,3))
print(A,'\n',B,'\n',C)
print(A[:,:3]*B)
print(dot(A[:,:3],diag(B)))

In [ ]:
arctan??

In [ ]:
u.shape,s.shape,vh.shape